# spark definiter gide orilly

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Structured API").getOrCreate()
spark

# 5 Basic Structured Operations

In [5]:
spark.read.format("json").load("./data/flight-data/json/2015-summary.json").schema

StructType(List(StructField(DEST_COUNTRY_NAME,StringType,true),StructField(ORIGIN_COUNTRY_NAME,StringType,true),StructField(count,LongType,true)))

In [6]:
# define mannully schme
from tokenize import String
from pyspark.sql.types import StructType,StructField,StringType,LongType

schema = StructType(
   [ StructField("DEST_COUNTRY_NAME",StringType(),True),
    StructField("ORIGIN_COUNTRY_NAME",StringType(),True),
    StructField("count",LongType(),True)
   ]
)

In [2]:
flight_df = spark.read \
    .format("json") \
    .load("./data/flight-data/json/2015-summary.json")
    

NameError: name 'spark' is not defined

In [9]:
flight_df.columns

['DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'count']

In [10]:
flight_df.first()

Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15)

In [11]:
from pyspark.sql import Row 

my_row = Row("heloo",None,1,False)


In [12]:
my_row[0]

'heloo'

In [16]:
# in Python
from pyspark.sql.functions import expr, col, column
flight_df.select(
expr("DEST_COUNTRY_NAME"),
col("DEST_COUNTRY_NAME"),
column("DEST_COUNTRY_NAME"))\
.show(2)

+-----------------+-----------------+-----------------+
|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|
+-----------------+-----------------+-----------------+
|    United States|    United States|    United States|
|    United States|    United States|    United States|
+-----------------+-----------------+-----------------+
only showing top 2 rows



In [13]:
# empty dataframe 
from pyspark.sql.types import StructField,StructType,StringType
emt_rdd = spark.sparkContext.emptyRDD()
col = StructType([])
df3 = spark.createDataFrame(emt_rdd,schema=col)
df3.show()


++
||
++
++



In [23]:
df_mt = spark.createDataFrame([],schema=StructType([]))
df_mt

DataFrame[]

In [25]:
emp_rdd = spark.sparkContext.emptyRDD()
col = StructType([StructField("name",StringType()),StructField("year",StringType())])
col2 = "name String, year int"
df4 = spark.createDataFrame(emp_rdd,col2)
df4.show()


df4.schema

+----+----+
|name|year|
+----+----+
+----+----+



StructType(List(StructField(name,StringType,true),StructField(year,IntegerType,true)))

In [17]:

# in Python
flight_df.selectExpr(
"*", # all original columns
"(DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME) as withinCountry")\
.show(2)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



In [18]:
from pyspark.sql.functions import lit 
flight_df.select(expr("*"),lit(1).alias("one")).show(2)

+-----------------+-------------------+-----+---+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|one|
+-----------------+-------------------+-----+---+
|    United States|            Romania|   15|  1|
|    United States|            Croatia|    1|  1|
+-----------------+-------------------+-----+---+
only showing top 2 rows



In [19]:
# add cloumns 
flight_df.withColumn("number_one",lit(1)).show(3)

+-----------------+-------------------+-----+----------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|number_one|
+-----------------+-------------------+-----+----------+
|    United States|            Romania|   15|         1|
|    United States|            Croatia|    1|         1|
|    United States|            Ireland|  344|         1|
+-----------------+-------------------+-----+----------+
only showing top 3 rows



In [20]:
# rename column
flight_df.withColumnRenamed("DEST_COUNTRY_NAME","dest").show(3)

+-------------+-------------------+-----+
|         dest|ORIGIN_COUNTRY_NAME|count|
+-------------+-------------------+-----+
|United States|            Romania|   15|
|United States|            Croatia|    1|
|United States|            Ireland|  344|
+-------------+-------------------+-----+
only showing top 3 rows



In [23]:
# changing the type of columns 
flight_df.withColumn("count2", col("count").cast("int"))

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint, count2: int]

In [26]:
flight_df.distinct().count()

256

In [34]:
# concataning and appending rows
schma = flight_df.schema
new_rows = [
    Row("New_country","other_country",5),
    Row("new coutnry2","other_country2",1)
]

new_df = spark.createDataFrame(new_rows,schema)

In [37]:
new_df.show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|      New_country|      other_country|    5|
|     new coutnry2|     other_country2|    1|
+-----------------+-------------------+-----+



In [42]:
flight_df.union(new_df).where("count=1").where(col("ORIGIN_COUNTRY_NAME")!= "United States").show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
|    United States|          Gibraltar|    1|
|    United States|             Cyprus|    1|
|    United States|            Estonia|    1|
|    United States|          Lithuania|    1|
|    United States|           Bulgaria|    1|
|    United States|            Georgia|    1|
|    United States|            Bahrain|    1|
|    United States|   Papua New Guinea|    1|
|    United States|         Montenegro|    1|
|    United States|            Namibia|    1|
|     new coutnry2|     other_country2|    1|
+-----------------+-------------------+-----+



In [45]:
flight_df.rdd.getNumPartitions()

1

In [46]:
# partitions
df_part=flight_df.repartition(5)
df_part.rdd.getNumPartitions()

5

In [47]:
flight_df.repartition(col("DEST_COUNTRY_NAME"))

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint]

In [49]:
collect_df = flight_df.limit(10)

In [50]:
collect_df.take(5)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344),
 Row(DEST_COUNTRY_NAME='Egypt', ORIGIN_COUNTRY_NAME='United States', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=62)]

In [51]:
collect_df.show(5,False)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|United States    |Romania            |15   |
|United States    |Croatia            |1    |
|United States    |Ireland            |344  |
|Egypt            |United States      |15   |
|United States    |India              |62   |
+-----------------+-------------------+-----+
only showing top 5 rows



In [52]:
collect_df.collect()

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344),
 Row(DEST_COUNTRY_NAME='Egypt', ORIGIN_COUNTRY_NAME='United States', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=62),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Singapore', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Grenada', count=62),
 Row(DEST_COUNTRY_NAME='Costa Rica', ORIGIN_COUNTRY_NAME='United States', count=588),
 Row(DEST_COUNTRY_NAME='Senegal', ORIGIN_COUNTRY_NAME='United States', count=40),
 Row(DEST_COUNTRY_NAME='Moldova', ORIGIN_COUNTRY_NAME='United States', count=1)]

In [5]:
spark.stop()

In [85]:
cl = collect_df.toLocalIterator()

In [30]:
flight_df.sample(False,0.5,5).count()

138

In [96]:
try :
    print(next(cl))
except StopIteration as r:
    print("no next item ",r)

no next item  


# ch.6 working with different types


- Booleans
- Numbers
- Strings
- Dates and timestamps
- Handling null
- Complex types
- User-defined functions

In [8]:
# read file
df = spark.read \
    .format("csv") \
    .option("header","true") \
    .option("inferschema","true") \
    .load("./data/retail-data/by-day/2010-12-01.csv")

df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [7]:
df.show(3,False)

+---------+---------+----------------------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                       |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |
+---------+---------+----------------------------------+--------+-------------------+---------+----------+--------------+
|536365   |85123A   |WHITE HANGING HEART T-LIGHT HOLDER|6       |2010-12-01 08:26:00|2.55     |17850.0   |United Kingdom|
|536365   |71053    |WHITE METAL LANTERN               |6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|
|536365   |84406B   |CREAM CUPID HEARTS COAT HANGER    |8       |2010-12-01 08:26:00|2.75     |17850.0   |United Kingdom|
+---------+---------+----------------------------------+--------+-------------------+---------+----------+--------------+
only showing top 3 rows



In [9]:
df2 = spark.read.options(inferschema=True,header=True).csv("./data/retail-data/by-day/2010-12-01.csv")

In [11]:
df2.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



## types
1. Lit 
2. Booleans -> and , or ,True, False 
3. instr
4. Numbers
5. row_number -> monotonically_increasing_id 
6. capital,lower,upper -> initcap,lower,upper 
7. strip -> ltrim,rtrim,trim,lpad,rpad 

In [13]:
# lit 
from pyspark.sql.functions import  lit 
df.select(lit(5),lit("five"),lit(5.0))

DataFrame[5: int, five: string, 5.0: double]

In [19]:
# boolean, where
from pyspark.sql.functions import col 
df.where(col("InvoiceNo")==536365) \
    .select("InvoiceNo","Description") \
    .show(8,False)

+---------+-----------------------------------+
|InvoiceNo|Description                        |
+---------+-----------------------------------+
|536365   |WHITE HANGING HEART T-LIGHT HOLDER |
|536365   |WHITE METAL LANTERN                |
|536365   |CREAM CUPID HEARTS COAT HANGER     |
|536365   |KNITTED UNION FLAG HOT WATER BOTTLE|
|536365   |RED WOOLLY HOTTIE WHITE HEART.     |
|536365   |SET 7 BABUSHKA NESTING BOXES       |
|536365   |GLASS STAR FROSTED T-LIGHT HOLDER  |
+---------+-----------------------------------+



In [20]:
# instr
from pyspark.sql.functions import instr
priceFilter = col("UnitPrice") >600
decriFilter = instr(df.Description,"POSTAGE") >=1

df.where(df.StockCode.isin("DOT")).where(priceFilter | decriFilter).show()

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|   Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|   536544|      DOT|DOTCOM POSTAGE|       1|2010-12-01 14:32:00|   569.77|      null|United Kingdom|
|   536592|      DOT|DOTCOM POSTAGE|       1|2010-12-01 17:06:00|   607.49|      null|United Kingdom|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+



In [23]:
df.withColumn("is_expensive",(col("StockCode")=="DOT") & (priceFilter | decriFilter)) \
    .where("is_expensive").select("unitPrice","is_expensive").show(5)

+---------+------------+
|unitPrice|is_expensive|
+---------+------------+
|   569.77|        true|
|   607.49|        true|
+---------+------------+



In [24]:
# null
df.where(col("Description").eqNullSafe("hello")).show()

+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
+---------+---------+-----------+--------+-----------+---------+----------+-------+



In [26]:
# numbers
from pyspark.sql.functions import  expr, pow

fabricatedQuantity = pow(col("Quantity")*col("UnitPrice"),2) + 5 
df.select(expr("CustomerID"),fabricatedQuantity.alias("realQuantity")).show(3)

+----------+------------------+
|CustomerID|      realQuantity|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
|   17850.0|             489.0|
+----------+------------------+
only showing top 3 rows



In [40]:
# round 
from pyspark.sql.functions import lit,round,bround

df.select(round(lit(2.76656)),bround(lit("1.4677"))).show(3)

+-----------------+-----------------+
|round(2.76656, 0)|bround(1.4677, 0)|
+-----------------+-----------------+
|              3.0|              1.0|
|              3.0|              1.0|
|              3.0|              1.0|
+-----------------+-----------------+
only showing top 3 rows



In [43]:
# corr -> Pearson correlation coefficient
from pyspark.sql.functions import  corr
df.stat.corr("Quantity","UnitPrice")
df.select(corr("Quantity","UnitPrice")).show()

+-------------------------+
|corr(Quantity, UnitPrice)|
+-------------------------+
|     -0.04112314436835551|
+-------------------------+



In [46]:
# decribie static parameter
df.describe().show()

+-------+-----------------+------------------+--------------------+------------------+-------------------+------------------+------------------+--------------+
|summary|        InvoiceNo|         StockCode|         Description|          Quantity|        InvoiceDate|         UnitPrice|        CustomerID|       Country|
+-------+-----------------+------------------+--------------------+------------------+-------------------+------------------+------------------+--------------+
|  count|             3108|              3108|                3098|              3108|               3108|              3108|              1968|          3108|
|   mean| 536516.684944841|27834.304044117645|                null| 8.627413127413128|               null| 4.151946589446603|15661.388719512195|          null|
| stddev|72.89447869788873|17407.897548583845|                null|26.371821677029203|               null|15.638659854603892|1854.4496996893627|          null|
|    min|           536365|             

In [50]:
df.stat.crosstab("StockCode","Quantity").show(3)

+------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|StockCode_Quantity| -1|-10|-12| -2|-24| -3| -4| -5| -6| -7|  1| 10|100| 11| 12|120|128| 13| 14|144| 15| 16| 17| 18| 19|192|  2| 20|200| 21|216| 22| 23| 24| 25|252| 27| 28|288|  3| 30| 32| 33| 34| 36|384|  4| 40|432| 47| 48|480|  5| 50| 56|  6| 60|600| 64|  7| 70| 72|  8| 80|  9| 96|
+------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|             22578|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0| 

In [53]:
 # row num
from pyspark.sql.functions import monotonically_increasing_id

df.select(monotonically_increasing_id()+1,"invoiceNo").show(3)

+-----------------------------------+---------+
|(monotonically_increasing_id() + 1)|invoiceNo|
+-----------------------------------+---------+
|                                  1|   536365|
|                                  2|   536365|
|                                  3|   536365|
+-----------------------------------+---------+
only showing top 3 rows



In [55]:
from pyspark.sql.functions import initcap
df.select(initcap(col("Description"))).show(5)

+--------------------+
|initcap(Description)|
+--------------------+
|White Hanging Hea...|
| White Metal Lantern|
|Cream Cupid Heart...|
|Knitted Union Fla...|
|Red Woolly Hottie...|
+--------------------+
only showing top 5 rows



In [59]:
from pyspark.sql.functions import lower,upper ,initcap 

df.select(col("Description"),lower(col("Description")),upper(col("Description"))).show()

+--------------------+--------------------+--------------------+
|         Description|  lower(Description)|  upper(Description)|
+--------------------+--------------------+--------------------+
|WHITE HANGING HEA...|white hanging hea...|WHITE HANGING HEA...|
| WHITE METAL LANTERN| white metal lantern| WHITE METAL LANTERN|
|CREAM CUPID HEART...|cream cupid heart...|CREAM CUPID HEART...|
|KNITTED UNION FLA...|knitted union fla...|KNITTED UNION FLA...|
|RED WOOLLY HOTTIE...|red woolly hottie...|RED WOOLLY HOTTIE...|
|SET 7 BABUSHKA NE...|set 7 babushka ne...|SET 7 BABUSHKA NE...|
|GLASS STAR FROSTE...|glass star froste...|GLASS STAR FROSTE...|
|HAND WARMER UNION...|hand warmer union...|HAND WARMER UNION...|
|HAND WARMER RED P...|hand warmer red p...|HAND WARMER RED P...|
|ASSORTED COLOUR B...|assorted colour b...|ASSORTED COLOUR B...|
|POPPY'S PLAYHOUSE...|poppy's playhouse...|POPPY'S PLAYHOUSE...|
|POPPY'S PLAYHOUSE...|poppy's playhouse...|POPPY'S PLAYHOUSE...|
|FELTCRAFT PRINCES...|fel

In [76]:
from pyspark.sql.functions import  lit,rtrim,ltrim,trim,lpad,rpad

str1= "      hello       "
str2 = "hello"
df.select(lpad(lit(str2),12,"#").alias("hello")).show(5)

+------------+
|       hello|
+------------+
|#######hello|
|#######hello|
|#######hello|
|#######hello|
|#######hello|
+------------+
only showing top 5 rows



## Regular Expression

1. regexp_replace(col,old_str,new_str)
2. regexp_extract(col,ext_str,1)
3. translate(col,"LEET","1337") -> PLET - P137
4. contains 
5. loacate



In [78]:
from pyspark.sql.functions import regexp_replace

reg_str = "BLACK|WHITE|RED|GREEEN|BLUE"

df.select(regexp_replace(col("Description"),reg_str,"COLOR").alias("color_clean"),col("Description")).show(6,False)

+-----------------------------------+-----------------------------------+
|color_clean                        |Description                        |
+-----------------------------------+-----------------------------------+
|COLOR HANGING HEART T-LIGHT HOLDER |WHITE HANGING HEART T-LIGHT HOLDER |
|COLOR METAL LANTERN                |WHITE METAL LANTERN                |
|CREAM CUPID HEARTS COAT HANGER     |CREAM CUPID HEARTS COAT HANGER     |
|KNITTED UNION FLAG HOT WATER BOTTLE|KNITTED UNION FLAG HOT WATER BOTTLE|
|COLOR WOOLLY HOTTIE COLOR HEART.   |RED WOOLLY HOTTIE WHITE HEART.     |
|SET 7 BABUSHKA NESTING BOXES       |SET 7 BABUSHKA NESTING BOXES       |
+-----------------------------------+-----------------------------------+
only showing top 6 rows



In [80]:
from pyspark.sql.functions import translate

df.select(translate(col("Description"),"LEET","1338"),col("Description")).show()

+----------------------------------+--------------------+
|translate(Description, LEET, 1338)|         Description|
+----------------------------------+--------------------+
|              WHI83 HANGING H3A...|WHITE HANGING HEA...|
|               WHI83 M38A1 1AN83RN| WHITE METAL LANTERN|
|              CR3AM CUPID H3AR8...|CREAM CUPID HEART...|
|              KNI883D UNION F1A...|KNITTED UNION FLA...|
|              R3D WOO11Y HO88I3...|RED WOOLLY HOTTIE...|
|              S38 7 BABUSHKA N3...|SET 7 BABUSHKA NE...|
|              G1ASS S8AR FROS83...|GLASS STAR FROSTE...|
|              HAND WARM3R UNION...|HAND WARMER UNION...|
|              HAND WARM3R R3D P...|HAND WARMER RED P...|
|              ASSOR83D CO1OUR B...|ASSORTED COLOUR B...|
|              POPPY'S P1AYHOUS3...|POPPY'S PLAYHOUSE...|
|              POPPY'S P1AYHOUS3...|POPPY'S PLAYHOUSE...|
|              F318CRAF8 PRINC3S...|FELTCRAFT PRINCES...|
|              IVORY KNI883D MUG...|IVORY KNITTED MUG...|
|             

In [85]:
from pyspark.sql.functions import regexp_extract
ext_str = "(BLACK|WHITE|RED|GREEN|BLUE)"
df.select(
    regexp_extract(col("Description"),ext_str,1).alias("color_clean"),
    col("Description")
).show(5,False)

+-----------+-----------------------------------+
|color_clean|Description                        |
+-----------+-----------------------------------+
|WHITE      |WHITE HANGING HEART T-LIGHT HOLDER |
|WHITE      |WHITE METAL LANTERN                |
|           |CREAM CUPID HEARTS COAT HANGER     |
|           |KNITTED UNION FLAG HOT WATER BOTTLE|
|RED        |RED WOOLLY HOTTIE WHITE HEART.     |
+-----------+-----------------------------------+
only showing top 5 rows



In [90]:
from pyspark.sql.functions import instr 
is_black = instr(col("Description"),"BLACK") >=1
is_white = instr(col("Description"),"WHITE") >=1

df.withColumn("has_simple_color", is_black | is_white ) \
    .where("has_simple_color") \
    .select("Description").show(10,False)

+----------------------------------+
|Description                       |
+----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER|
|WHITE METAL LANTERN               |
|RED WOOLLY HOTTIE WHITE HEART.    |
|WHITE HANGING HEART T-LIGHT HOLDER|
|WHITE METAL LANTERN               |
|WOOD 2 DRAWER CABINET WHITE FINISH|
|WOOD S/3 CABINET ANT WHITE FINISH |
|WOODEN PICTURE FRAME WHITE FINISH |
|WOODEN FRAME ANTIQUE WHITE        |
|RED WOOLLY HOTTIE WHITE HEART.    |
+----------------------------------+
only showing top 10 rows



In [105]:
from pyspark.sql.functions import expr, locate 
simple_color = ["black","white","red","green", "blue"]

def color_locator(col,color_str1):
    return locate(color_str1.upper(),col).cast("boolean").alias("is_"+ color_str1)

selected_col = [color_locator(df.Description,c) for c in simple_color]
selected_col.append(expr("*"))

df.select(*selected_col) \
    .where(expr("is_white OR is_red")).\
    select("Description").show(3)

+--------------------+
|         Description|
+--------------------+
|WHITE HANGING HEA...|
| WHITE METAL LANTERN|
|RED WOOLLY HOTTIE...|
+--------------------+
only showing top 3 rows



## Date and timestaps 



In [ ]:
1. 




In [107]:
df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [112]:
from pyspark.sql.functions import current_date,current_timestamp

date_df = spark.range(10) \
    .withColumn("today",current_date()) \
    .withColumn("now",current_timestamp())

date_df.createOrReplaceTempView("datetb")

In [118]:
date_df.show(20,False)

+---+----------+-----------------------+
|id |today     |now                    |
+---+----------+-----------------------+
|0  |2022-10-12|2022-10-12 12:09:02.835|
|1  |2022-10-12|2022-10-12 12:09:02.835|
|2  |2022-10-12|2022-10-12 12:09:02.835|
|3  |2022-10-12|2022-10-12 12:09:02.835|
|4  |2022-10-12|2022-10-12 12:09:02.835|
|5  |2022-10-12|2022-10-12 12:09:02.835|
|6  |2022-10-12|2022-10-12 12:09:02.835|
|7  |2022-10-12|2022-10-12 12:09:02.835|
|8  |2022-10-12|2022-10-12 12:09:02.835|
|9  |2022-10-12|2022-10-12 12:09:02.835|
+---+----------+-----------------------+



In [119]:
spark.sql("select * from datetb").show()

+---+----------+--------------------+
| id|     today|                 now|
+---+----------+--------------------+
|  0|2022-10-12|2022-10-12 12:09:...|
|  1|2022-10-12|2022-10-12 12:09:...|
|  2|2022-10-12|2022-10-12 12:09:...|
|  3|2022-10-12|2022-10-12 12:09:...|
|  4|2022-10-12|2022-10-12 12:09:...|
|  5|2022-10-12|2022-10-12 12:09:...|
|  6|2022-10-12|2022-10-12 12:09:...|
|  7|2022-10-12|2022-10-12 12:09:...|
|  8|2022-10-12|2022-10-12 12:09:...|
|  9|2022-10-12|2022-10-12 12:09:...|
+---+----------+--------------------+



In [121]:
from pyspark.sql.functions import date_add, date_sub
date_df.select(date_sub(col("today"),5),date_add(col("today"),5)).show()

+------------------+------------------+
|date_sub(today, 5)|date_add(today, 5)|
+------------------+------------------+
|        2022-10-07|        2022-10-17|
|        2022-10-07|        2022-10-17|
|        2022-10-07|        2022-10-17|
|        2022-10-07|        2022-10-17|
|        2022-10-07|        2022-10-17|
|        2022-10-07|        2022-10-17|
|        2022-10-07|        2022-10-17|
|        2022-10-07|        2022-10-17|
|        2022-10-07|        2022-10-17|
|        2022-10-07|        2022-10-17|
+------------------+------------------+



In [123]:
# diffreence betewwn two dates
from pyspark.sql.functions import datediff,months_between, to_date 
date_df.withColumn("week_ago",date_sub(col("today"),7))\
    .withColumn("day_diff",datediff(col("week_ago"),col("today"))).show(5,False)


+---+----------+----------------------+----------+--------+
|id |today     |now                   |week_ago  |day_diff|
+---+----------+----------------------+----------+--------+
|0  |2022-10-12|2022-10-12 12:14:06.46|2022-10-05|-7      |
|1  |2022-10-12|2022-10-12 12:14:06.46|2022-10-05|-7      |
|2  |2022-10-12|2022-10-12 12:14:06.46|2022-10-05|-7      |
|3  |2022-10-12|2022-10-12 12:14:06.46|2022-10-05|-7      |
|4  |2022-10-12|2022-10-12 12:14:06.46|2022-10-05|-7      |
+---+----------+----------------------+----------+--------+
only showing top 5 rows



In [124]:
date_df.select(
    to_date(lit("2016-01-01")).alias("start"),
    to_date(lit("2017-05-22")).alias("end")
).select(months_between(col("start"),col("end"))).show()

+--------------------------------+
|months_between(start, end, true)|
+--------------------------------+
|                    -16.67741935|
|                    -16.67741935|
|                    -16.67741935|
|                    -16.67741935|
|                    -16.67741935|
|                    -16.67741935|
|                    -16.67741935|
|                    -16.67741935|
|                    -16.67741935|
|                    -16.67741935|
+--------------------------------+



In [131]:
date_df.select(to_date(lit("13-12-2016"))).show()

+---------------------+
|to_date('13-12-2016')|
+---------------------+
|                 null|
|                 null|
|                 null|
|                 null|
|                 null|
|                 null|
|                 null|
|                 null|
|                 null|
|                 null|
+---------------------+



In [144]:
dateFormat = "yyyy-dd-MM"
cleanDateDF = spark.range(1).select(
to_date(lit("2017-12-13"), "yyyy-MM-dd").alias("date"),
to_date(lit("2017-20-12"), dateFormat).alias("date2"))
cleanDateDF.createOrReplaceTempView("dateTable2")

In [143]:
cleanDateDF.show(5)

+----------+----------+
|      date|     date2|
+----------+----------+
|2017-01-13|2017-12-20|
+----------+----------+



In [137]:
 # coalesce returns the frist not null values from columns
from pyspark.sql.functions import coalesce 
df.select(coalesce(col("Description"), col("CustomerId"))).show()

+---------------------------------+
|coalesce(Description, CustomerId)|
+---------------------------------+
|             WHITE HANGING HEA...|
|              WHITE METAL LANTERN|
|             CREAM CUPID HEART...|
|             KNITTED UNION FLA...|
|             RED WOOLLY HOTTIE...|
|             SET 7 BABUSHKA NE...|
|             GLASS STAR FROSTE...|
|             HAND WARMER UNION...|
|             HAND WARMER RED P...|
|             ASSORTED COLOUR B...|
|             POPPY'S PLAYHOUSE...|
|             POPPY'S PLAYHOUSE...|
|             FELTCRAFT PRINCES...|
|             IVORY KNITTED MUG...|
|             BOX OF 6 ASSORTED...|
|             BOX OF VINTAGE JI...|
|             BOX OF VINTAGE AL...|
|             HOME BUILDING BLO...|
|             LOVE BUILDING BLO...|
|             RECIPE BOX WITH M...|
+---------------------------------+
only showing top 20 rows



In [158]:
# ifnull,nullif, nvl, nvl2
spark.sql(
    """select ifnull(null,'return_value'), nullif('value','value'),
        nvl(null, 'return_value'),
        nvl2('not_null', 'return_value', "else_value")
"""
).show()



+----------------------------+------------------------+-------------------------+----------------------------------------------+
|ifnull(NULL, 'return_value')|nullif('value', 'value')|nvl(NULL, 'return_value')|nvl2('not_null', 'return_value', 'else_value')|
+----------------------------+------------------------+-------------------------+----------------------------------------------+
|                return_value|                    null|             return_value|                                  return_value|
+----------------------------+------------------------+-------------------------+----------------------------------------------+



In [160]:
#d drop
df.na.drop()
df.na.drop("any")
df.na.drop("all", subset=["StockCode", "InvoiceNo"])

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: double, CustomerID: double, Country: string]

In [ ]:
# fill
df.na.fill("vaule_string")
df.na.fill("all", subset=["StockCode", "InvoiceNo"])


fill_cols_vals = {"StockCode": 5, "Description" : "No Value"}
df.na.fill(fill_cols_vals)

In [161]:
df.na.replace([""], ["UNKNOWN"], "Description")

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: double, CustomerID: double, Country: string]

## struct

In [165]:
from pyspark.sql.functions import  struct

comlex_df = df.select(struct("Description","InvoiceNo").alias("complex"))

In [167]:
comlex_df.printSchema()

root
 |-- complex: struct (nullable = false)
 |    |-- Description: string (nullable = true)
 |    |-- InvoiceNo: string (nullable = true)



## ARRAY

In [169]:
from pyspark.sql.functions import split
df.select(split(col("Description")," ")).show(3)

+-------------------------+
|split(Description,  , -1)|
+-------------------------+
|     [WHITE, HANGING, ...|
|     [WHITE, METAL, LA...|
|     [CREAM, CUPID, HE...|
+-------------------------+
only showing top 3 rows



In [171]:
df.select(split(col("Description")," ").alias("array_col")).select(col("array_col")[0]).show()

+------------+
|array_col[0]|
+------------+
|       WHITE|
|       WHITE|
|       CREAM|
|     KNITTED|
|         RED|
|         SET|
|       GLASS|
|        HAND|
|        HAND|
|    ASSORTED|
|     POPPY'S|
|     POPPY'S|
|   FELTCRAFT|
|       IVORY|
|         BOX|
|         BOX|
|         BOX|
|        HOME|
|        LOVE|
|      RECIPE|
+------------+
only showing top 20 rows



In [173]:
# length
from pyspark.sql.functions import size 
df.select(size(split(col("Description")," "))).show(3)

+-------------------------------+
|size(split(Description,  , -1))|
+-------------------------------+
|                              5|
|                              3|
|                              5|
+-------------------------------+
only showing top 3 rows



In [174]:
# array contains
from pyspark.sql.functions import array_contains
df.select(array_contains(split(col("Description")," "),"WHITE")).show(3)

+------------------------------------------------+
|array_contains(split(Description,  , -1), WHITE)|
+------------------------------------------------+
|                                            true|
|                                            true|
|                                           false|
+------------------------------------------------+
only showing top 3 rows



In [177]:
# explode
from pyspark.sql.functions import explode
df.withColumn("splitted",split(col("Description")," ")) \
    .withColumn("exploded",explode(col("splitted"))) \
        .select("Description","InvoiceNo","exploded").show(20)

+--------------------+---------+--------+
|         Description|InvoiceNo|exploded|
+--------------------+---------+--------+
|WHITE HANGING HEA...|   536365|   WHITE|
|WHITE HANGING HEA...|   536365| HANGING|
|WHITE HANGING HEA...|   536365|   HEART|
|WHITE HANGING HEA...|   536365| T-LIGHT|
|WHITE HANGING HEA...|   536365|  HOLDER|
| WHITE METAL LANTERN|   536365|   WHITE|
| WHITE METAL LANTERN|   536365|   METAL|
| WHITE METAL LANTERN|   536365| LANTERN|
|CREAM CUPID HEART...|   536365|   CREAM|
|CREAM CUPID HEART...|   536365|   CUPID|
|CREAM CUPID HEART...|   536365|  HEARTS|
|CREAM CUPID HEART...|   536365|    COAT|
|CREAM CUPID HEART...|   536365|  HANGER|
|KNITTED UNION FLA...|   536365| KNITTED|
|KNITTED UNION FLA...|   536365|   UNION|
|KNITTED UNION FLA...|   536365|    FLAG|
|KNITTED UNION FLA...|   536365|     HOT|
|KNITTED UNION FLA...|   536365|   WATER|
|KNITTED UNION FLA...|   536365|  BOTTLE|
|RED WOOLLY HOTTIE...|   536365|     RED|
+--------------------+---------+--

In [181]:
# maps 
from pyspark.sql.functions import create_map

complex_map = df.select(create_map(col("Description"),col("InvoiceNo")).alias("complex_map"))
complex_map.show(2,False)

+----------------------------------------------+
|complex_map                                   |
+----------------------------------------------+
|[WHITE HANGING HEART T-LIGHT HOLDER -> 536365]|
|[WHITE METAL LANTERN -> 536365]               |
+----------------------------------------------+
only showing top 2 rows



In [182]:
complex_map.selectExpr("complex_map['WHITE METAL LANTERN']").show(5)

+--------------------------------+
|complex_map[WHITE METAL LANTERN]|
+--------------------------------+
|                            null|
|                          536365|
|                            null|
|                            null|
|                            null|
+--------------------------------+
only showing top 5 rows



In [183]:
complex_map.select(explode(col('complex_map'))).show(5)

+--------------------+------+
|                 key| value|
+--------------------+------+
|WHITE HANGING HEA...|536365|
| WHITE METAL LANTERN|536365|
|CREAM CUPID HEART...|536365|
|KNITTED UNION FLA...|536365|
|RED WOOLLY HOTTIE...|536365|
+--------------------+------+
only showing top 5 rows



In [184]:
complex_map.select(explode('complex_map')).show(2)

+--------------------+------+
|                 key| value|
+--------------------+------+
|WHITE HANGING HEA...|536365|
| WHITE METAL LANTERN|536365|
+--------------------+------+
only showing top 2 rows



## JSON

In [186]:
json_df = spark.range(1).selectExpr(
    """ '{"my_json_key":{
        "my_json_value":[1,2,3] }}' as json_string """
)

In [188]:
json_df.show()

+--------------------+
|         json_string|
+--------------------+
|{"my_json_key":{
...|
+--------------------+



In [191]:
from pyspark.sql.functions import get_json_object, json_tuple
json_df.select(
    get_json_object(col("json_string"), "$.my_json_key.my_json_value[1]"),
    json_tuple(col("json_string"),"my_json_key")
).show()

+------------------------------------------------------------+--------------------+
|get_json_object(json_string, $.my_json_key.my_json_value[1])|                  c0|
+------------------------------------------------------------+--------------------+
|                                                           2|{"my_json_value":...|
+------------------------------------------------------------+--------------------+



In [193]:
from pyspark.sql.functions import to_json
df.selectExpr("(InvoiceNo, Description) as my_struct")\
    .select(to_json(col("my_struct"))).show(3,False)

+-------------------------------------------------------------------------+
|to_json(my_struct)                                                       |
+-------------------------------------------------------------------------+
|{"InvoiceNo":"536365","Description":"WHITE HANGING HEART T-LIGHT HOLDER"}|
|{"InvoiceNo":"536365","Description":"WHITE METAL LANTERN"}               |
|{"InvoiceNo":"536365","Description":"CREAM CUPID HEARTS COAT HANGER"}    |
+-------------------------------------------------------------------------+
only showing top 3 rows



In [195]:
from pyspark.sql.functions import from_json
from pyspark.sql.types import StringType,StructField,StructType

parse_schema = StructType(
    (
        StructField("InvoiceNo",StringType(),True),
        StructField("Description",StringType(),True)
    )
)

df.selectExpr("(InvoiceNo, Description) as my_struct") \
    .select(to_json(col("my_struct")).alias("new_json"))\
        .select(from_json(col("new_json"),parse_schema),col("new_json")).show(4,False)

+---------------------------------------------+--------------------------------------------------------------------------+
|from_json(new_json)                          |new_json                                                                  |
+---------------------------------------------+--------------------------------------------------------------------------+
|[536365, WHITE HANGING HEART T-LIGHT HOLDER] |{"InvoiceNo":"536365","Description":"WHITE HANGING HEART T-LIGHT HOLDER"} |
|[536365, WHITE METAL LANTERN]                |{"InvoiceNo":"536365","Description":"WHITE METAL LANTERN"}                |
|[536365, CREAM CUPID HEARTS COAT HANGER]     |{"InvoiceNo":"536365","Description":"CREAM CUPID HEARTS COAT HANGER"}     |
|[536365, KNITTED UNION FLAG HOT WATER BOTTLE]|{"InvoiceNo":"536365","Description":"KNITTED UNION FLAG HOT WATER BOTTLE"}|
+---------------------------------------------+--------------------------------------------------------------------------+
only showing top

In [ ]:
## UDF



In [211]:
# power 3 
from pyspark.sql.functions import udf
udf_df = spark.range(1,6).toDF("num")
udf_df.show()

def power3(val):
    return val**3

pow3_udf = udf(power3)

udf_df.select(pow3_udf(col("num"))).show()





+---+
|num|
+---+
|  1|
|  2|
|  3|
|  4|
|  5|
+---+

+-----------+
|power3(num)|
+-----------+
|          1|
|          8|
|         27|
|         64|
|        125|
+-----------+



In [214]:
spark.udf.register("power3r",power3)

<function __main__.power3(val)>

In [215]:
udf_df.selectExpr("power3r(num)").show(2)

+------------+
|power3r(num)|
+------------+
|           1|
|           8|
+------------+
only showing top 2 rows



In [1]:
# def power3(number:Double):Double = number * number * number

In [26]:
spark.stop()

################ end ##############